In [1]:
import pandas as pd
from common.clean import replace
from common.heuristics import has_op, test_for_suitable

In [2]:
df = pd.DataFrame(pd.read_csv("../data/eqs_100k.tsv", sep = "\t", header=None))
df.columns = ["eq_id", "eq"]

In [3]:
df.head()

,eq_id,eq
0,EQDS578Q,\\begin{equation}\\end{equation}
1,EQDS33191905Q,\\begin{equation}\n \\det\\psi_-^{-1}=m_+^{(2)...
2,EQDS9271890Q,\\begin{equation}T^+_{\\varepsilon}(r):=\\inf\...
3,EQDS14066999Q,\\begin{equation}\\max_{U \\hbox{ is unitary }...
4,EQDS45956134Q,\\begin{equation}\\alpha=\\frac{1}{\\sum_{i=1}...


In [4]:
df['clean'] = replace(df['eq'])

In [5]:
def find_aligned(s):
    if 'aligned' in s:
        return True
    else: return False

In [6]:
df_alignments = df[df['clean'].apply(find_aligned)].reset_index()

In [7]:
from common.heuristics import split_high_level_eqs
df_alignments['clean_split'] = df_alignments['clean'].apply(split_high_level_eqs)

In [8]:
se = df_alignments['clean_split'].apply(lambda x: len(x) > 1 if x is not None else False)
df_alignments = df_alignments[se].reset_index()

In [61]:
se = df_alignments['clean_split'].apply(test_for_suitable)
df_alignments['clean_split_filtered'] = se
filt = df_alignments['clean_split_filtered'].apply(lambda x: x is not None)
df_alignments = df_alignments[filt]

In [62]:
from common.heuristics import edit_distance
def filter_edit_distance(li):
    ret = []
    if len(li) < 2:
        return None
    prev = li[0]
    for el in li[1:]:
        prev_len = len(prev)
        next_len = len(el)
        dist =  edit_distance(prev, el) / next_len if (prev_len > 0) and (next_len > 0) else 0
        if dist > 0.5:
            ret.append((prev, el, dist))
        prev = el
    
    return ret if len(ret) > 0 else None
        

In [63]:
se = df_alignments['clean_split_filtered'].apply(lambda x: [e.replace('\\begin{aligned}', '').replace('\\end{aligned}', '')\
                                                   for e in x])\
                                 .apply(filter_edit_distance)

In [76]:
se[se.apply(lambda x: x is not None)].iloc[40]

[('\\lim\\limits_{\\eta\\o 0}\\frac{T^{\\eta,1}}{\\ln\\frac{1}{\\epsilon}}',
  ':I_1,\\\\\\lim\\limits_{\\eta\\o 0}\\frac{T^{\\eta,2}-\\widetilde T^{\\eta,1}}{\\ln\\frac{1}{\\epsilon}}',
  0.6813186813186813),
 (':I_1,\\\\\\lim\\limits_{\\eta\\o 0}\\frac{T^{\\eta,2}-\\widetilde T^{\\eta,1}}{\\ln\\frac{1}{\\epsilon}}',
  ':I_2-I_1,\\\\\\lim\\limits_{\\eta\\o 0}\\frac{T^{\\eta,0}-\\widetilde T^{\\eta,2}}{\\ln\\frac{1}{\\epsilon}}',
  0.9368421052631579)]